# Landsat 7 ETM Reflectancia Superficie google Earth Engine 

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP con geeMap

In [2]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [3]:
ee_zona = geemap.shp_to_ee(r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\ZONA_geo.shp")

In [4]:
type(ee_zona)

ee.featurecollection.FeatureCollection

In [7]:
# Visualizar mapa
geometria = ee_zona.geometry()
Map.centerObject(geometria, 10)
Map.addLayer(ee_zona, {"color" : "blue"}, "zona")
Map

Map(bottom=140852.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['posit…

## Coleccion de Landsat 7 ETM RS

In [8]:
L7_RS = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")\
          .filterDate('2000-01-01','2001-12-31')\
          .filterMetadata('CLOUD_COVER','less_than',20)\
          .filterBounds(ee_zona)

In [9]:
# Conteo de imagenes
count = L7_RS.size().getInfo()
print("Cantidad de imagenes L7_RS:", count)

Cantidad de imagenes L7_RS: 5


In [10]:
# Imprimir la lista de ID Imagenes
ID_L7 = L7_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_L7)

['LE07_002069_20010722', 'LE07_003069_20000523', 'LE07_003069_20000726', 'LE07_003069_20000827', 'LE07_003069_20010830']


In [11]:
# Extraer la ffecha de imagencollection
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [12]:
fecha = list_coll(L7_RS).getInfo()
print(fecha)

['2001/07/22', '2000/05/23', '2000/07/26', '2000/08/27', '2001/08/30']


In [13]:
# Imprimir la lista Porcentaje Nubosidad
porcentaje_nube = L7_RS.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER"]).get("list").getInfo()
print(porcentaje_nube)

[6, 3, 8, 16, 13]


In [14]:
# Imprimir la lista Path y Row
WRS_PATH = L7_RS.reduceColumns(ee.Reducer.toList(),["WRS_PATH"]).get("list").getInfo()
WRS_ROW = L7_RS.reduceColumns(ee.Reducer.toList(),["WRS_ROW"]).get("list").getInfo()
print(WRS_PATH)
print(WRS_ROW)

[2, 3, 3, 3, 3]
[69, 69, 69, 69, 69]


In [16]:
# Crear un diccionario con las listas
dic = {"ID": ID_L7, "Fecha": fecha,"Porcentaje Nube": porcentaje_nube,"PATH": WRS_PATH,"ROW":WRS_ROW}
dic

{'ID': ['LE07_002069_20010722',
  'LE07_003069_20000523',
  'LE07_003069_20000726',
  'LE07_003069_20000827',
  'LE07_003069_20010830'],
 'Fecha': ['2001/07/22',
  '2000/05/23',
  '2000/07/26',
  '2000/08/27',
  '2001/08/30'],
 'Porcentaje Nube': [6, 3, 8, 16, 13],
 'PATH': [2, 3, 3, 3, 3],
 'ROW': [69, 69, 69, 69, 69]}

In [17]:
import pandas as pd

In [18]:
# Crear un DataFrame mediante el diccionario
df1 = pd.DataFrame(dic)
print(df1)
print("Tipo de clase: ", type(dic))

                     ID       Fecha  Porcentaje Nube  PATH  ROW
0  LE07_002069_20010722  2001/07/22                6     2   69
1  LE07_003069_20000523  2000/05/23                3     3   69
2  LE07_003069_20000726  2000/07/26                8     3   69
3  LE07_003069_20000827  2000/08/27               16     3   69
4  LE07_003069_20010830  2001/08/30               13     3   69
Tipo de clase:  <class 'dict'>


## Exportar tabla ID, Fecha, Porcentaje Nubosidad

In [20]:
import os

In [21]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [22]:
df1.to_csv("file_name_L7.csv") # relative position

## Seleccion imagen Landsat 7

In [25]:
# Importar imagen Landsat 7 RS 
image = ee.Image("LANDSAT/LE07/C01/T1_SR" + "/" + ID_L7[1])

In [26]:
# Nombre de las bandas
print(image.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']


## Seleccionar Bandas

In [27]:
# Crear nombre por bandas
bandas_RS = ["B1","B2","B3","B4","B5","B7"]
bandas_TB = ["B6"]

In [28]:
# Seleccionar imagenes por bandas
img_L7_RS = image.select(bandas_RS)
img_L7_TB = image.select(bandas_TB)

In [29]:
# Nombre de las bandas
print("Banda RS:",img_L7_RS.bandNames().getInfo())
print("Banda TB:",img_L7_TB.bandNames().getInfo())

Banda RS: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
Banda TB: ['B6']


## Verificar proyeccion

In [30]:
# Consultar Proyeccion
print("Proyeccion RS:",img_L7_RS.projection().getInfo())
print("Proyeccion TB:",img_L7_TB.projection().getInfo())

Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 223485, 0, -30, -1334385]}
Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 223485, 0, -30, -1334385]}


## Metadato landsat

In [31]:
# Nombres del Metadato
img_L7_RS.propertyNames().getInfo()

['IMAGE_QUALITY',
 'SATELLITE',
 'SOLAR_AZIMUTH_ANGLE',
 'CLOUD_COVER',
 'system:id',
 'WRS_PATH',
 'EARTH_SUN_DISTANCE',
 'system:time_start',
 'LANDSAT_ID',
 'SENSING_TIME',
 'ESPA_VERSION',
 'SOLAR_ZENITH_ANGLE',
 'system:footprint',
 'system:version',
 'WRS_ROW',
 'GEOMETRIC_RMSE_MODEL_Y',
 'LEVEL1_PRODUCTION_DATE',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL_X',
 'system:asset_size',
 'GEOMETRIC_RMSE_MODEL',
 'SR_APP_VERSION',
 'PIXEL_QA_VERSION',
 'system:index',
 'system:bands',
 'system:band_names']

In [32]:
# Extraer valor metadato distancia del sol a tierra
d = img_L7_RS.get("EARTH_SUN_DISTANCE").getInfo()
d

1.012644

In [33]:
# Extraer angulo zenital solar
SOLAR_ZENITH_ANGLE = img_L7_RS.get("SOLAR_ZENITH_ANGLE").getInfo()
SOLAR_ZENITH_ANGLE

44.813087

In [36]:
import math
SOLAR_ZENITH_ANGLE_R = SOLAR_ZENITH_ANGLE*math.pi/180
SOLAR_ZENITH_ANGLE_R

0.7821359161326681

In [37]:
# Determinacion angulo elevacion solar radianes
SUN_ELEVATION = 90 - SOLAR_ZENITH_ANGLE
SUN_ELEVATION_R = SUN_ELEVATION*math.pi/180
print("Elevacion solar radianes:", SUN_ELEVATION_R)

Elevacion solar radianes: 0.7886604106622284


## Visualizar Landsat

In [38]:
# Simbologia estala 0 - 10000
viz_RS = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 250,
    'max': 7000,
    'gamma': 1.90
}

In [39]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [40]:
# Convertir en Geometria
geometria = ee_zona.geometry()
Map.centerObject(geometria,8)

In [41]:
# Visualizar ESCALADO 0 - 10000
Map.addLayer(img_L7_RS, viz_RS,name = "Landsat7 RS")
Map

Map(center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['position', 'transparen…

## Factor de escala Reflectancia Superficie

In [44]:
# Escalar la imagen Landsat 5 RS 0 - 1
img_L7_RS_Es = img_L7_RS.multiply(0.0001)

In [42]:
# Simbologia estala 0 - 1
viz_es = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [45]:
# Visualizar ESCALADO 0 - 1
Map.addLayer(img_L7_RS_Es, viz_es,name = "Landsat7 RS Escalado")
Map

Map(bottom=35438.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['positi…

## Factor de escala Temperatura Brillo

In [46]:
# Escalar la imagen Landsat 7 TB convertir Kelvin a Celsius
img_L7_TB_es = img_L7_TB.subtract(273.15).multiply(0.01)

In [47]:
# Crear su simbologia TB
viz_tb = {
    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'],
    'min': 25,
    'max': 28
}

In [48]:
# Visualizacion temperatura brillo
Map.addLayer(img_L7_TB_es, viz_tb,name = "TB B6 Celsius")
Map

Map(bottom=35364.0, center=[-12.511665400971019, -69.52972412109376], controls=(WidgetControl(options=['positi…

## Reproyectar y recortar segun zona

In [49]:
# Reproyectar segun zona de estudio
img_L7_TB_es = img_L7_TB_es.reproject(crs="EPSG:32719", scale=30)
img_L7_RS_Es = img_L7_RS_Es.reproject(crs="EPSG:32719", scale=30)

In [50]:
# Recortar RS y TB
img_L7_TB_es_clip = img_L7_TB_es.clip(ee_zona)
img_L7_RS_Es_clip = img_L7_RS_Es.clip(ee_zona)

In [51]:
# Consultar Proyeccion
print("Proyeccion TB:",img_L7_TB_es_clip.projection().getInfo())
print("Proyeccion RS:",img_L7_RS_Es_clip.projection().getInfo())

Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}


In [52]:
img_L7_RS_Es_clip.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B7']

In [53]:
# Visualizacion TB y RS
Map.addLayer(img_L7_TB_es_clip, viz_tb,name = "TB B6 clip")
Map.addLayer(img_L7_RS_Es_clip, viz_es,name = "L7 RS clip")
Map

Map(bottom=35364.0, center=[-12.511665400971019, -69.52972412109376], controls=(WidgetControl(options=['positi…

## Exportar Imagen calibrado

In [54]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [55]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [56]:
# Exportar imagen Temperatura de brillo B6
geemap.ee_export_image(img_L7_TB_es_clip, filename="L7_TB_es_clip.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L7_TB_es_clip.tif


In [57]:
# Exportar imagen Reflectancia superficie B543
geemap.ee_export_image(img_L7_RS_Es_clip.select(["B5","B4","B3"]), filename="L7_RS_Es_clip.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L7_RS_Es_clip.tif


In [58]:
# Exportar imagen Reflectancia superficie B543 Bandas separadas
geemap.ee_export_image(img_L7_RS_Es_clip.select(["B5","B4","B3"]), filename="L7_RS_Es_clip_.tif", scale=30, region=geometria, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE


In [59]:
# Exportar en google drive
geemap.ee_export_image_to_drive(img_L7_RS_Es_clip, description='L7_RS_Es_clip', folder='GEE_Python', region=geometria, scale=30)

Exporting L7_RS_Es_clip ...
